In [1]:
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("seaborn")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1603073737723_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/87/a6/8d7d06f6b69236a3c1818157875ceb1259ba0d9df4194f4fe138ffdc0f41/matplotlib-3.3.2-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/5e/c4/6c4fe722df5343c33226f0b4e0bb042e4dc13483228b4718baf286f86d87/certifi-2020.6.20-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-2.4.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9f/b7/4b3304e5fd986e2ad8a0157adba88c01dcefc111deaf84a37175af5f5e43/Pillow-8.0.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached htt

In [2]:
# import libraries
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import udf, max, min, lit, when, col, count, countDistinct
from pyspark.sql.types import *


import math
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, GBTClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# create a Spark session
spark = SparkSession.builder.appName("EDA  Sparkify").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
path = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
user_log = spark.read.json(path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def check_and_clean_column(df, column):
    ### Check for anomalies in the dataset
    df.select([column]).dropDuplicates().sort(column).show()
    
    ### Filter for empty string, and print results
    print(df.count())
    df = df.filter(df[column] != "") #on the mini data 
    print(df.count())
    
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
user_log = check_and_clean_column(user_log, 'userId')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
| userId|
+-------+
|1000025|
|1000035|
|1000083|
|1000103|
|1000164|
|1000168|
|1000182|
|1000194|
|1000214|
|1000233|
|1000244|
|1000248|
|1000280|
|1000353|
|1000407|
|1000409|
|1000446|
|1000503|
|1000527|
|1000611|
+-------+
only showing top 20 rows

26259199
26259199

In [7]:
user_log = check_and_clean_column(user_log, 'userAgent')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           userAgent|
+--------------------+
|                null|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
|"Mozilla/5.0 (Mac...|
+--------------------+
only showing top 20 rows

26259199
25480720

In [8]:
user_log = check_and_clean_column(user_log, 'location')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|            location|
+--------------------+
|        Aberdeen, SD|
|        Aberdeen, WA|
|         Abilene, TX|
|             Ada, OK|
|          Adrian, MI|
|           Akron, OH|
|      Alamogordo, NM|
|          Albany, GA|
|          Albany, OR|
|Albany-Schenectad...|
|       Albemarle, NC|
|      Albert Lea, MN|
|     Albertville, AL|
|     Albuquerque, NM|
|      Alexandria, LA|
|      Alexandria, MN|
|           Alice, TX|
|Allentown-Bethleh...|
|            Alma, MI|
|          Alpena, MI|
+--------------------+
only showing top 20 rows

25480720
25480720

In [9]:
# add churn, 1 or 0
churned = udf(lambda x: int(x=="Cancellation Confirmation"), IntegerType())
user_log = user_log.withColumn("Churned", churned("page"))
    
# add churn user flag
user_log = user_log.withColumn("userChurned", max('Churned').over(Window.partitionBy('UserId')))
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
### Create necessary functions for UDFs

def get_platform(x):
    if 'compatible' in x:
        return 'Windows'
    elif 'iPad' in x:
        return 'iPad'
    elif 'iPhone' in x:
        return 'iPhone'
    elif 'Macintosh' in x:
        return 'Mac'
    elif 'Windows NT 5.1' in x:
        return 'Windows'
    elif 'Windows NT 6.0' in x:
        return 'Windows'
    elif 'Windows NT 6.1' in x:
        return 'Windows'
    elif 'Windows NT 6.2' in x:
        return 'Windows'
    elif 'Windows NT 6.3' in x:
        return 'Windows'
    elif 'X11' in x:
        return 'Linux'
    
def get_browser(x):
    if 'Firefox' in x:
        return 'Firefox'
    elif 'Safari' in x:
        if 'Chrome' in x:
            return 'Chrome'
        else:
            return 'Safari'
    elif 'Trident' in x:
        return 'IE'
    else:
        return np.NaN
    
### Create UDFs, and add columns with them
get_browser_udf = udf(get_browser, StringType())
get_platform_udf = udf(get_platform, StringType())

# Adding Columns
user_log = user_log.withColumn( 'browser', get_browser_udf(user_log.userAgent) )
user_log = user_log.withColumn( 'platform', get_platform_udf(user_log.userAgent) )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
### Doing the same with location data
def get_state(x):
    return x.split(', ')[1]

def get_city(x):
    return x.split(', ')[0]

get_state_udf = udf(get_state, StringType())
get_city_udf = udf(get_city, StringType())

user_log = user_log.withColumn( 'state', get_state_udf(user_log.location) )
user_log = user_log.withColumn( 'city', get_city_udf(user_log.location) )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
### Create unique day column from timestamp
get_hour = udf(lambda x: math.floor( (x / 1000.0) / (60*60*24) ) )
user_log = user_log.withColumn("uniqueDay", get_hour(user_log.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
total_daily_sessions_df = user_log.select('userId', 'sessionId', 'uniqueDay').groupby(['userId', 'uniqueDay']).agg(count('sessionId').alias('totalDailySessions')).sort('userId')
avg_daily_sessions_df = total_daily_sessions_df.groupBy('userId').avg('totalDailySessions').withColumnRenamed('avg(totalDailySessions)', 'avgDailySessions')

user_log = user_log.join(avg_daily_sessions_df, on='userId')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def create_avg_daily_page_interaction_column(df, page_interaction, page_title):
    total_daily_df = df.select('userId', 'page', col('uniqueDay').alias('uniqueDay')).groupby(['userId', 'uniqueDay']).agg(count(when(col("page") == page_interaction, 1)).alias('totalDaily' + page_title)).sort('userId')
    avg_daily_df = total_daily_df.groupBy('userId').avg('totalDaily' + page_title).withColumnRenamed('avg(totalDaily' + page_title + ')', 'avgDaily' + page_title)

    df = df.join(avg_daily_df, on='userId')
    
    return df

user_log = create_avg_daily_page_interaction_column(user_log, 'Thumbs Up', 'ThumbsUp')
user_log = create_avg_daily_page_interaction_column(user_log, 'Thumbs Down', 'ThumbsDown')
user_log = create_avg_daily_page_interaction_column(user_log, 'Error', 'Errors')
user_log = create_avg_daily_page_interaction_column(user_log, 'Roll Advert', 'Advertisements')
user_log = create_avg_daily_page_interaction_column(user_log, 'NextSong', 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
total_distinct_artists_df = user_log.select('userId', 'artist').groupby('userId').agg(countDistinct('artist')).withColumnRenamed('count(DISTINCT artist)', 'totalDistinctArtists')
user_log = user_log.join(total_distinct_artists_df, on='userId')


duration_using_service_df = user_log.select('userId', 'uniqueDay').groupby('userId').agg(max('uniqueDay') - min('uniqueDay')).withColumnRenamed('(max(uniqueDay) - min(uniqueDay))', 'durationUsingService')
user_log = user_log.join(duration_using_service_df, on='userId')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
user_log = user_log.drop('artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'location', 'method', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'Churned', 'uniqueDay', 'page')

user_log = user_log.dropDuplicates(['userId'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
def transform_features_pipeline(df):
    ### Create pipeline for processing data
    # Categorical Data
    pipeline_stages = [StringIndexer(inputCol=column, outputCol=column+"Index") 
                        for column in ['level','browser', 'platform', 'state', 'city'] ]

    # Numerical Data
    assembler = VectorAssembler(inputCols=['avgDailySessions', 'avgDailyThumbsUp', 'avgDailyThumbsDown', 'avgDailyErrors', 'avgDailyAdvertisements', 'avgDailyNextSong', 'totalDistinctArtists', 'durationUsingService'], outputCol="features")
    pipeline_stages.append(assembler)

    standard_scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')
    pipeline_stages.append(standard_scaler)

    # Categorical and Numerical Data
    combined_assembler = VectorAssembler(inputCols=['scaledFeatures','levelIndex', 'browserIndex', 'platformIndex', 'stateIndex', 'cityIndex'], outputCol='combinedFeatures')

    pipeline_stages.append(combined_assembler)

    process_pipeline = Pipeline(stages=pipeline_stages)
    
    ### Fit and transform the df into features
    
    df_model = process_pipeline.fit(df).transform(df)

    df_model = df_model.select('userChurned','combinedFeatures').withColumnRenamed('userChurned', 'label')
    
    return df_model
   
    
user_log_model = transform_features_pipeline(user_log)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
train, test = user_log_model.randomSplit([0.8, 0.2], seed=42)

train.cache()
test.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[label: int, combinedFeatures: vector]

In [19]:
f1_evaluator = MulticlassClassificationEvaluator(metricName='f1')
auc_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')

def check_model_f1(model, train, test):
    fitted_model = model.fit(train)
    model_results = fitted_model.transform(test)
    
    f1_score = f1_evaluator.evaluate(model_results.select(col('label'), col('prediction')))
    
    print('The F1 score on the test set is {:.2%}'.format(f1_score)) 
    
    roc_value = auc_evaluator.evaluate(model_results)
    
    print('The areaUnderROC on the test set is {:.2%}'.format(roc_value))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
log_reg = LogisticRegression(featuresCol="combinedFeatures", labelCol="label", maxIter=10)
gbt = GBTClassifier(featuresCol="combinedFeatures", labelCol="label", maxIter=10, maxBins=820)
rf = RandomForestClassifier(featuresCol="combinedFeatures", labelCol="label", maxBins=820)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
check_model_f1(log_reg, train, test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-21:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1081



The F1 score on the test set is 81.51%
The areaUnderROC on the test set is 85.16%

In [25]:
check_model_f1(gbt, train, test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The F1 score on the test set is 81.07%
The areaUnderROC on the test set is 81.73%

In [26]:
check_model_f1(rf, train, test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The F1 score on the test set is 81.57%
The areaUnderROC on the test set is 83.53%

Exception in thread cell_monitor-25:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 6164

Exception in thread cell_monitor-26:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][sta

In [27]:
log_reg = LogisticRegression(featuresCol="combinedFeatures", labelCol="label")

log_reg_param_grid = ParamGridBuilder().addGrid(log_reg.elasticNetParam, [0, 0.5]).addGrid(log_reg.maxIter, [10, 50]).build()
cross_validator = CrossValidator(estimator = log_reg, estimatorParamMaps = log_reg_param_grid, evaluator = MulticlassClassificationEvaluator(metricName='f1'), numFolds = 3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
cv_model = cross_validator.fit(train)
cv_test_results = cv_model.transform(test)
cv_train_results = cv_model.transform(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-28:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 8320



In [29]:
f1_score_train = f1_evaluator.evaluate(cv_test_results.select(col('label'), col('prediction')))
print('The F1 score on the training set for the hypertuned log model is {:.2%}'.format(f1_score_train)) 

f1_score_test = f1_evaluator.evaluate(cv_train_results.select(col('label'), col('prediction')))
print('The F1 score on the test set for the hypertuned log model is {:.2%}'.format(f1_score_test))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-29:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 29365



The F1 score on the training set for the hypertuned log model is 81.95%
The F1 score on the test set for the hypertuned log model is 82.45%

In [30]:
roc_train_value = auc_evaluator.evaluate(cv_train_results)
print('The areaUnderROC on the training set for the hypertuned log model is {:.2%}'.format(roc_train_value))

roc_test_value = auc_evaluator.evaluate(cv_test_results)
print('The areaUnderROC on the test set for the hypertuned log model is {:.2%}'.format(roc_test_value))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-30:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 29640



The areaUnderROC on the training set for the hypertuned log model is 86.64%
The areaUnderROC on the test set for the hypertuned log model is 85.50%

In [36]:
cv_feature_vals = cv_model.bestModel.coefficients
feature_cols = ['avgDailySessions', 'avgDailyThumbsUp', 'avgDailyThumbsDown', 'avgDailyErrors', 'avgDailyAdvertisements', 'avgDailyNextSong', 'totalDistinctArtists', 'durationUsingService','levelIndex', 'browserIndex', 'platformIndex', 'stateIndex', 'cityIndex']

for i in range(len(cv_feature_vals)):
        print("{} : {} \n".format(feature_cols[i], cv_feature_vals[i]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

avgDailySessions : 6.335622184463661 

avgDailyThumbsUp : -0.8189511859489664 

avgDailyThumbsDown : 0.23961139428787867 

avgDailyErrors : -0.04392872602093854 

avgDailyAdvertisements : -0.04826264838585036 

avgDailyNextSong : -5.820138373437149 

totalDistinctArtists : 0.9154048395391323 

durationUsingService : -1.5543157554359075 

levelIndex : 0.29792354028200796 

browserIndex : -0.0016389255515044938 

platformIndex : -0.012566220799818957 

stateIndex : 0.0015495141766243639 

cityIndex : -3.9506386894644574e-05

In [37]:
rf = RandomForestClassifier(featuresCol="combinedFeatures", labelCol="label", maxBins=820)

rf_param_grid = ParamGridBuilder().addGrid(rf.maxDepth, [10, 20]).addGrid(rf.impurity, ['entropy', 'gini']).build()
cross_validator = CrossValidator(estimator = rf, estimatorParamMaps = rf_param_grid, evaluator = MulticlassClassificationEvaluator(metricName='f1'), numFolds = 3)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
cv_model = cross_validator.fit(train)
cv_test_results = cv_model.transform(test)
cv_train_results = cv_model.transform(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-38:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 30352

Exception in thread cell_monitor-39:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][st

In [39]:
f1_score_train = f1_evaluator.evaluate(cv_test_results.select(col('label'), col('prediction')))
print('The F1 score on the training set for the hypertuned log model is {:.2%}'.format(f1_score_train)) 

f1_score_test = f1_evaluator.evaluate(cv_train_results.select(col('label'), col('prediction')))
print('The F1 score on the test set for the hypertuned log model is {:.2%}'.format(f1_score_test))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The F1 score on the training set for the hypertuned log model is 84.70%
The F1 score on the test set for the hypertuned log model is 91.89%

In [40]:
roc_train_value = auc_evaluator.evaluate(cv_train_results)
print('The areaUnderROC on the training set for the hypertuned log model is {:.2%}'.format(roc_train_value))

roc_test_value = auc_evaluator.evaluate(cv_test_results)
print('The areaUnderROC on the test set for the hypertuned log model is {:.2%}'.format(roc_test_value))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The areaUnderROC on the training set for the hypertuned log model is 96.14%
The areaUnderROC on the test set for the hypertuned log model is 86.93%

In [42]:
cv_feature_vals = cv_model.bestModel.featureImportances
feature_cols = ['avgDailySessions', 'avgDailyThumbsUp', 'avgDailyThumbsDown', 'avgDailyErrors', 'avgDailyAdvertisements', 'avgDailyNextSong', 'totalDistinctArtists', 'durationUsingService','levelIndex', 'browserIndex', 'platformIndex', 'stateIndex', 'cityIndex']

for i in range(len(cv_feature_vals)):
        print("{} : {} \n".format(feature_cols[i], cv_feature_vals[i]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

avgDailySessions : 0.033863868772272133 

avgDailyThumbsUp : 0.01908649314784476 

avgDailyThumbsDown : 0.04055437744868638 

avgDailyErrors : 0.014889069380147812 

avgDailyAdvertisements : 0.06219234578975993 

avgDailyNextSong : 0.022332058458700576 

totalDistinctArtists : 0.11935425931490011 

durationUsingService : 0.4177032729430631 

levelIndex : 0.0025212672051650458 

browserIndex : 0.003194875665667809 

platformIndex : 0.0026145482192327218 

stateIndex : 0.039308792209264413 

cityIndex : 0.22238477144529542